# Métodos ensemble:

1. Cargar los datos MNIST y dividirlos en un conjunto de entrenamiento, un conjunto de validación y un conjunto de test (por ejemplo, utilizar 50.000 instancias para entrenamiento, 10.000 para validación y 10.000 para pruebas). Luego, entrenar varios clasificadores, como un clasificador Random Forest, un clasificador Extra-Trees y un clasificador SVM. A continuación, intentar combinarlos en un conjunto que supere a cada clasificador individual en el conjunto de validación, utilizando votación soft o hard. Una vez que haya encontrado uno, probarlo en el conjunto de pruebas. ¿Cuánto mejor se desempeña en comparación con los clasificadores individuales?


In [13]:
from sklearn.datasets import fetch_openml

# Obtengo dataset MNIST
mnist = fetch_openml('mnist_784')
X, y = mnist["data"], mnist["target"]


/home/fran/Documentos/5 AÑO/Sistemas Inteligentes carpeta/Sistemas-Inteligentes/sappienv/lib/python3.8/site-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [14]:
from sklearn.model_selection import train_test_split

# Divido en conjuntos de entrenamiento
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=10000, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=10000, random_state=42)

In [15]:
from sklearn.ensemble import RandomForestClassifier

# Entrenamos el clasificador RandomForest
rfc = RandomForestClassifier(n_estimators=30, random_state=42) #30 estimators porque sino tarda mucho
rfc.fit(X_train, y_train)


RandomForestClassifier(n_estimators=30, random_state=42)

In [16]:
from sklearn.ensemble import ExtraTreesClassifier

# Entrenamos el clasificador ExtraTree
etc = ExtraTreesClassifier(n_estimators=30, random_state=42)
etc.fit(X_train, y_train)

ExtraTreesClassifier(n_estimators=30, random_state=42)

In [17]:
from sklearn.svm import LinearSVC

# Entrenamos el clasificador Linear SVC
svc = LinearSVC()
svc.fit(X_train, y_train)

/home/fran/Documentos/5 AÑO/Sistemas Inteligentes carpeta/Sistemas-Inteligentes/sappienv/lib/python3.8/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC()

In [19]:
# Entrenamos el modelo Gradient Boosting para luego usarlo en el clasificador de votacion, ya que el SVC nos arruina el modelo

#from sklearn.ensemble import GradientBoostingClassifier
#grd = GradientBoostingClassifier()
#grd.fit(X_train, y_train)

# tarda demasiado en ejecutarse

In [ ]:
# Predecimos y evaluamos en el conjunto de validacion cada uno de los modelos para ver como se desempeñam los Clasificadores individuales

from sklearn.metrics import accuracy_score

# RandomForest
y_pred_rfc = rfc.predict(X_valid)
accuracy = accuracy_score(y_valid, y_pred_rfc)

print("Accuracy: ", accuracy)


Accuracy:  0.9625


In [ ]:
# ExtraTree
y_pred_etc = etc.predict(X_valid)
accuracy = accuracy_score(y_valid, y_pred_etc)

print("Accuracy: ", accuracy)

Accuracy:  0.9652


In [ ]:
# LinearSVC
y_pred_svc = svc.predict(X_valid)
accuracy = accuracy_score(y_valid, y_pred_svc)

print("Accuracy: ", accuracy)

Accuracy:  0.8779


In [20]:
from sklearn.metrics import accuracy_score
from sklearn.ensemble import VotingClassifier

# Utilizamos clasificador Voting Clasiffier "hard"

voting_clf=VotingClassifier(
    estimators=[('rfc',rfc),('etc',etc)],voting='hard') #evaluamos solo con dos modelos porque SVC arruina el modelo y Gradient Boosting tarda mucho (+1 hora)

# Entrenamos y predecimos el modelo
voting_clf.fit(X_train,y_train)
y_pred_voting_clf = voting_clf.predict(X_valid)

# Analizamos Accuracy
accuracy_voting=accuracy_score(y_valid, y_pred_voting_clf)
print("Accuracy: ",accuracy_voting)

Accuracy:  0.963


In [21]:
# Utilizamos clasificador Voting Clasiffier "soft"

voting_clf=VotingClassifier(
    estimators=[('rfc',rfc),('etc',etc)],voting='soft') #evaluamos solo con dos modelos porque SVC arruina el modelo y Gradient Boosting tarda mucho (+1 hora)

# Entrenamos y predecimos el modelo
voting_clf.fit(X_train,y_train)
y_pred_voting_clf = voting_clf.predict(X_valid)

# Analizamos Accuracy
accuracy_voting=accuracy_score(y_valid, y_pred_voting_clf)
print("Accuracy: ",accuracy_voting)

Accuracy:  0.9678


In [22]:
from sklearn.metrics import accuracy_score

# Evaluar el conjunto combinado en el conjunto de pruebas
y_pred_voting_clf = voting_clf.predict(X_test)
voting_accuracy = accuracy_score(y_test, y_pred_voting_clf)

# Evaluar cada clasificador individual en el conjunto de pruebas
y_pred_rfc = rfc.predict(X_test)
rfc_accuracy = accuracy_score(y_test, y_pred_rfc)

y_pred_etc = etc.predict(X_test)
etc_accuracy = accuracy_score(y_test, y_pred_etc)

y_pred_svc = svc.predict(X_test)
svc_accuracy = accuracy_score(y_test, y_pred_svc)

# Mostrar los resultados
print("Accuracy del conjunto combinado (votación soft): {:.2f}%".format(voting_accuracy * 100))
print("Accuracy del clasificador Random Forest: {:.2f}%".format(rfc_accuracy * 100))
print("Accuracy del clasificador Extra-Trees: {:.2f}%".format(etc_accuracy * 100))
print("Accuracy del clasificador Gradient Boosting: {:.2f}%".format(svc_accuracy * 100))


Accuracy del conjunto combinado (votación suave): 96.54%
Accuracy del clasificador Random Forest: 95.95%
Accuracy del clasificador Extra-Trees: 96.33%
Accuracy del clasificador Gradient Boosting: 85.15%


2. Ejecutar los clasificadores individuales del ejercicio anterior para hacer predicciones en el conjunto de validación y crear un nuevo conjunto de entrenamiento con las predicciones resultantes: cada instancia de entrenamiento es un vector que contiene el conjunto de predicciones de todos los clasificadores para una imagen, y el objetivo es la clase de la imagen. Entrenar un clasificador en este nuevo conjunto de entrenamiento. ¡Felicidades, acaba de entrenar un blender, y junto con los clasificadores forma un conjunto de stacking! Ahora evaluar el conjunto en el conjunto de pruebas. Para cada imagen en el conjunto de pruebas, hacer predicciones con todos los clasificadores, y luego alimentar las predicciones al mezclador para obtener las predicciones del conjunto. ¿Cómo se compara con el clasificador de votación que entrenó anteriormente?

In [23]:
# Utilizamos las predicciones de los clasificadores individuales para crear el Blender donde creamos el nuevo conjunto de entrenamiento
import numpy as np

X_train_blender = np.column_stack((y_pred_rfc, y_pred_etc))
y_train_blender = y_valid

In [25]:
# Entrenamos un clasificador en el nuevo conjunto de entrenamiento

blender = RandomForestClassifier(n_estimators=100, random_state=42)
blender.fit(X_train_blender, y_train_blender)

TypeError: float() argument must be a string or a number, not 'DecisionTreeClassifier'



3. Realice el ejercicio 1. otra vez utilizando los algoritmo XGBoost, LightGBM y CatBoost.

In [ ]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier

# Cargar el dataset
digits = load_digits()

# Dividir los datos en conjuntos de entrenamiento, validación y prueba
X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)



<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216 from C header, got 232 from PyObject


In [ ]:
# Entrenar clasificador XGBoost
xgb_clf = xgb.XGBClassifier()
xgb_clf.fit(X_train, y_train)

# Entrenar clasificador LightGBM
lgb_clf = lgb.LGBMClassifier()
lgb_clf.fit(X_train, y_train)

# Entrenar clasificador CatBoost
cat_clf = CatBoostClassifier()
cat_clf.fit(X_train, y_train, verbose=False)



In [ ]:
# Predecir en el conjunto de validación
xgb_val_preds = xgb_clf.predict(X_val)
lgb_val_preds = lgb_clf.predict(X_val)
cat_val_preds = cat_clf.predict(X_val)




In [ ]:
# Evaluar la precisión de los clasificadores individuales en el conjunto de validación
xgb_val_acc = accuracy_score(y_val, xgb_val_preds)
lgb_val_acc = accuracy_score(y_val, lgb_val_preds)
cat_val_acc = accuracy_score(y_val, cat_val_preds)

print('XGBoost validation accuracy:', xgb_val_acc)
print('LightGBM validation accuracy:', lgb_val_acc)
print('CatBoost validation accuracy:', cat_val_acc)


XGBoost validation accuracy: 0.9666666666666667
LightGBM validation accuracy: 0.9805555555555555
CatBoost validation accuracy: 0.9888888888888889


In [ ]:

# Realizar votación soft en el conjunto de validación
val_preds = (xgb_clf.predict_proba(X_val) + lgb_clf.predict_proba(X_val) + cat_clf.predict_proba(X_val)) / 3
val_preds = val_preds.argmax(axis=1)
ensemble_val_acc = accuracy_score(y_val, val_preds)
print('Ensemble validation accuracy:', ensemble_val_acc)

# Realizar votación soft en el conjunto de prueba
test_preds = (xgb_clf.predict_proba(X_test) + lgb_clf.predict_proba(X_test) + cat_clf.predict_proba(X_test)) / 3
test_preds = test_preds.argmax(axis=1)
ensemble_test_acc = accuracy_score(y_test, test_preds)
print('Ensemble test accuracy:', ensemble_test_acc)

Ensemble validation accuracy: 0.9861111111111112
Ensemble test accuracy: 0.9805555555555555


# Reducción dimensional

1. Cargue el conjunto de datos MNIST (introducido en el capítulo 3) y divídalo en un conjunto de entrenamiento y un conjunto de pruebas (tome las primeras 60,000 instancias para entrenamiento y las 10,000 restantes para test). Entrene un clasificador Random Forest en el conjunto de datos y tome el tiempo que tarda, luego evalúe el modelo resultante en el conjunto de test. A continuación, use PCA para reducir la dimensionalidad del conjunto de datos, con una relación de varianza explicada del 95%. Entrenar un nuevo clasificador Random Forest en el conjunto de datos reducido y ver cuánto tiempo tarda. ¿Fue el entrenamiento mucho más rápido? A continuación, evalúe el clasificador en el conjunto de pruebas. ¿Cómo se compara con el clasificador anterior?

2. Use t-SNE para reducir el conjunto de datos MNIST a dos dimensiones y grafique el resultado usando Matplotlib. Puede usar un gráfico de dispersión utilizando 10 colores diferentes para representar la clase objetivo de cada imagen. Alternativamente, puede reemplazar cada punto en el gráfico de dispersión con la clase correspondiente de la instancia (un dígito del 0 al 9), o incluso graficar versiones reducidas de las imágenes de dígitos en sí mismas (si grafica todos los dígitos, la visualización será demasiado desordenada, por lo que debe dibujar una muestra aleatoria o graficar una instancia solo si no se ha graficado otra instancia a una distancia cercana). Debería obtener una visualización con grupos de dígitos bien separados. Intente usar otros algoritmos de reducción de dimensionalidad como PCA, LLE o MDS y compare las visualizaciones resultantes.

# Preguntas teóricas
### Métodos de ensamble

1. Si ha entrenado cinco modelos diferentes en los mismos datos de entrenamiento y todos logran una precisión del 95%, ¿existe alguna posibilidad de combinar estos modelos para obtener mejores resultados? Si es así, ¿cómo? Si no, ¿por qué?
2. ¿Cuál es la diferencia entre los clasificadores de votación hard y de votación soft?
3. ¿Es posible acelerar el entrenamiento de un conjunto de bagging distribuyéndolo en varios servidores? ¿Qué pasa con los conjuntos de pasting, los conjuntos de boosting, los Random Forest o los ensambles Stacking?
4. ¿Cuál es el beneficio de la evaluación out-of-bag (OOB)?
5. ¿Qué hace que los Extra-Trees sean más aleatorios que los Random Forest regulares? ¿Cómo puede esta aleatoriedad adicional ayudar? ¿Son los Extra-Trees más lentos o más rápidos que los Random Forest regulares?
6. Si su conjunto de AdaBoost no se ajusta lo suficientemente bien a los datos de entrenamiento, ¿qué hiperparámetros debe ajustar y cómo?
7. Si su conjunto de Gradient Boosting sobreajusta el conjunto de entrenamiento, ¿debería aumentar o disminuir la tasa de aprendizaje?


### Reducción dimensional

1. What are the main motivations for reducing a dataset’s dimensionality? What are
the main drawbacks?
2. What is the curse of dimensionality?
3. Once a dataset’s dimensionality has been reduced, is it possible to reverse the
operation? If so, how? If not, why?
4. Can PCA be used to reduce the dimensionality of a highly nonlinear dataset?
5. Suppose you perform PCA on a 1,000-dimensional dataset, setting the explained
variance ratio to 95%. How many dimensions will the resulting dataset have?
6. In what cases would you use vanilla PCA, Incremental PCA, Randomized PCA,
or Kernel PCA?
7. How can you evaluate the performance of a dimensionality reduction algorithm
on your dataset?
8. Does it make any sense to chain two different dimensionality reduction algo‐
rithms?